# Q1

In [276]:
import numpy as np
import json
from tqdm import tqdm

train_data = np.load('train_data.npy')

class HiddenMarkovModel():
    def __init__(self, initial_probabilities, transition_probs, emission_probs):
        self.transition_probs = transition_probs
        self.emission_probs = emission_probs
        self.initial_probabilities = initial_probabilities
        self._log_transition_probs = self._compute_log_probs(transition_probs)
        self._log_emission_probs = self._compute_log_probs(emission_probs)
        if not self._valid_hmm(): raise ValueError("not a valid markov model")

    def baum_welch(self, seq, iterations=100):
        for itr in tqdm(xrange(iterations)):
            initial_update = dict([(s, 0.0) for s in self.transition_probs])
            transition_update1 = dict(
                [(s, dict([(s, 0) for s in self.transition_probs])) for s in self.transition_probs])
            transition_update2 = dict([(s, 0) for s in self.transition_probs])
            emission_update1 = dict([(s, dict([(o, 0) for o in self.emission_probs[s]])) for s in self.emission_probs])
            emission_update2 = dict([(s, dict([(o, 0) for o in self.emission_probs[s]])) for s in self.emission_probs])

            for s in seq:
                init, tr1, tr2, em1, em2 = self._baum_welch_iteration(s)
                for s in self.transition_probs:
                    initial_update[s] += init[s]
                for s in self.transition_probs:
                    for x in self.transition_probs[s]:
                        transition_update1[s][x] += tr1[s][x]

                for s in self.transition_probs:
                    transition_update2[s] += tr2[s]

                for s in self.emission_probs:
                    for o in self.emission_probs[s]:
                        emission_update1[s][o] += em1[s][o]
                        emission_update2[s][o] += em2[s][o]

            self.initial_probabilities = dict([(s, (1 / len(seq)) * initial_update[s]) for s in self.transition_probs])
            for i in self.transition_probs:
                for j in self.transition_probs[i]:
                    self.transition_probs[i][j] = (transition_update1[i][j] / transition_update2[i])

            for i in self.emission_probs:
                for o in self.emission_probs[i]:
                    self.emission_probs[i][o] = (emission_update1[i][o] / emission_update2[i][o])

            self._log_transition_probs = self._compute_log_probs(self.transition_probs)
            self._log_emission_probs = self._compute_log_probs(self.emission_probs)


    def _baum_welch_iteration(self, seq, cons=100):
        forward_probabilities = [dict([(s, 0.0) for s in self.transition_probs]) for _ in xrange(len(seq))]
        eps = 0.00000000000000001
        forward_probabilities[0] = dict(
            [(s, np.log(self.initial_probabilities[s] + eps) + self._log_emission_probs[s][seq[0]]) for s in
             self.transition_probs])

        for i in xrange(1, len(seq)):
            for j in self.transition_probs:
                a = self._log_emission_probs[j][seq[i]]
                b = self._log_sum_exp([forward_probabilities[i - 1][k] + self._log_transition_probs[k][j] for k in
                                       self.transition_probs[j]])
                forward_probabilities[i][j] = a + b

        backward_probabilities = [dict([(s, 0.0) for s in self.transition_probs]) for _ in xrange(len(seq))]

        for j in self.transition_probs:
            backward_probabilities[len(seq) - 1][j] = 0

        for i in range(len(seq) - 2, -1, -1):
            for j in self.transition_probs:
                backward_probabilities[i][j] = self._log_sum_exp([
                    backward_probabilities[i + 1][k] + self._log_transition_probs[j][k] + self._log_emission_probs[k][
                        seq[i + 1]]
                    for k in self.transition_probs[j]
                ])
        gamma = [dict([(s, 0) for s in self.transition_probs]) for t in xrange(len(seq))]
        for t in xrange(len(seq)):
            norm = sum(
                [np.exp(forward_probabilities[t][k] + backward_probabilities[t][k] + 1200) for k in
                 self.transition_probs])
            for j in self.transition_probs:
                gamma[t][j] = np.exp(forward_probabilities[t][j] + backward_probabilities[t][j] + 1200) / norm

        zeta = [dict([(s, dict([(s, 0) for s in self.transition_probs])) for s in self.transition_probs]) for i in
                xrange(len(seq))]
        for t in xrange(len(seq) - 1):
            norm = sum([sum([
                np.exp(forward_probabilities[t][i] + self._log_transition_probs[i][j]
                       + backward_probabilities[t + 1][j] + self._log_emission_probs[j][seq[t + 1]] + 1200)
                for j in self.transition_probs[i]]) for i in self.transition_probs])

            for i in self.transition_probs:
                for j in self.transition_probs:
                    if j in self._log_transition_probs[i]:

                        zeta[t][i][j] = np.exp(forward_probabilities[t][i] + self._log_transition_probs[i][j]
                                               + backward_probabilities[t + 1][j] + self._log_emission_probs[j][
                                                   seq[t + 1]] + 1200) / norm
                    else:
                        zeta[t][i][j] = 0

        init = dict([(s, gamma[0][s]) for s in self.transition_probs])
        trans_prob1 = {}
        trans_prob2 = {}
        for i in self.transition_probs:
            trans_prob2[i] = 0
            for t in range(len(seq) - 1):
                trans_prob2[i] += gamma[t][i]

        for i in self.transition_probs:
            trans_prob1[i] = {}
            for j in self.transition_probs[i]:
                trans_prob1[i][j] = 0
                for t in range(len(seq) - 1):
                    trans_prob1[i][j] += zeta[t][i][j]

        em_prob1 = {}
        for i in self.emission_probs:
            em_prob1[i] = {}
            for o in self.emission_probs[i]:
                em_prob1[i][o] = sum([(seq[t] == o) * gamma[t][i] for t in xrange(len(seq))])

        em_prob2 = {}
        for i in self.emission_probs:
            em_prob2[i] = {}
            for o in self.emission_probs[i]:
                em_prob2[i][o] = sum([gamma[t][i] for t in xrange(len(seq))])

        return init, trans_prob1, trans_prob2, em_prob1, em_prob2

    def _log_sum_exp(self, arr):
        a = np.max(arr)
        # print(arr)
        return a + np.log(np.sum([np.exp(x - a) for x in arr]))

    def _compute_log_probs(self, prob_matrix):
        log_prob_matrix = {}
        for i in prob_matrix:
            log_prob_matrix[i] = {}
            for j in prob_matrix[i]:
                log_prob_matrix[i][j] = np.log(prob_matrix[i][j] + 0.00000000001)
        return log_prob_matrix

    def _valid_hmm(self):
        if (self.transition_probs != None) and (self.emission_probs != None):
            valid = True
            # print("1",set(self.transition_probs.keys()), set(self.emission_probs.keys()))
            valid &= (set(self.transition_probs.keys()) == set(self.emission_probs.keys()))

            return valid
        else:
            return True
        
    def likelihood(self, seq, log=True):
        alpha = [dict([(s, 0) for s in self.transition_probs]) for _ in xrange(len(seq))]
        alpha[0] = dict([(i, np.log(self.initial_probabilities[i]) + self._log_emission_probs[i][seq[0]]) for i in
                         self.emission_probs])
        for i in xrange(1, len(seq)):
            for j in self.transition_probs:
                alpha[i][j] = self._log_emission_probs[j][seq[i]] + self._log_sum_exp([
                    alpha[i - 1][k] + self._log_transition_probs[k][j] for k in self.transition_probs[j]])

        if log:
            return self._log_sum_exp(list(alpha[-1].values()))
        else:
            return sum([np.exp(p) for p in alpha[-1].values()])  # return the sum of the last row

    def backward_likelihood(self, seq, log=True):
        backward_probabilities = [dict([(s, 0.0) for s in self.transition_probs]) for _ in xrange(len(seq))]

        for j in self.transition_probs:
            backward_probabilities[len(seq) - 1][j] = 0

        for i in range(len(seq) - 2, -1, -1):
            for j in self.transition_probs:
                backward_probabilities[i][j] = self._log_sum_exp([
                    backward_probabilities[i + 1][k] + self._log_transition_probs[j][k] + self._log_emission_probs[k][
                        seq[i + 1]]
                    for k in self.transition_probs[j]
                ])

        if log:
            return self._log_sum_exp([(backward_probabilities[0][k] + self._log_emission_probs[k][seq[0]] + np.log(
                self.initial_probabilities[k])) for k in self.transition_probs])
        else:
            # return sum([np.exp(p) for p in alpha[-1].values()])  # return the sum of the last row
            return

    def viterbi(self, seq):
        T = [dict([(s, 0) for s in self.transition_probs]) for _ in xrange(len(seq))]
        T[0] = dict([(i, np.log(self.initial_probabilities[i]) + self._log_emission_probs[i][seq[0]]) for i in
                     self.emission_probs])
        T_bp = [dict([(s, 0) for s in self.transition_probs]) for _ in xrange(len(seq))]
        T_bp[0] = dict([(i, i) for i in self.emission_probs])

        for i in xrange(1, len(seq)):
            for j in self.transition_probs:
                T_bp[i][j] = max(self.transition_probs[j],
                                 key=lambda k: T[i - 1][k] + self._log_transition_probs[k][j] +
                                               self._log_emission_probs[j][seq[i]])

                prev_state = T_bp[i][j]
                T[i][j] = T[i - 1][prev_state] + self._log_transition_probs[prev_state][j] + self._log_emission_probs[j][seq[i]]

        # construct the most likely sequence
        state_sequence = [None for i in xrange(len(seq))]
        state_sequence[-1] = max(self.transition_probs,
                                 key=lambda k: T[-1][k])

        for i in xrange(len(seq) - 1):
            i = len(seq) - 2 - i  # go in reverse order
            state_sequence[i] = T_bp[i + 1][state_sequence[i + 1]]

        return state_sequence


transition = {
    0: {1: 0.5, 5: 0.5},
    1: {0: 0.5, 2: 0.5},
    2: {1: 0.5, 3: 0.5},
    3: {2: (1 / 3), 4: (1 / 3), 8: (1 / 3)},
    4: {3: 0.5, 9: 0.5},
    5: {0: (1 / 3), 6: (1 / 3), 10: (1 / 3)},
    6: {5: 0.25, 7: 0.25, 11: 0.25, 12: 0.25},
    7: {6: (1 / 3), 8: (1 / 3), 12: (1 / 3)},
    8: {3: 0.2, 7: 0.2, 9: 0.2, 12: 0.2, 13: 0.2},
    9: {8: (1 / 3), 4: (1 / 3), 14: (1 / 3)},
    10: {5: 0.5, 15: 0.5},
    11: {6: (1 / 3), 12: (1 / 3), 16: (1 / 3)},
    12: {6: 0.125, 7: 0.125, 8: 0.125, 11: 0.125, 13: 0.125, 16: 0.125, 17: 0.125, 18: 0.125},
    13: {8: 0.25, 12: 0.25, 14: 0.25, 18: 0.25},
    14: {9: (1 / 3), 13: (1 / 3), 19: (1 / 3)},
    15: {10: 0.5, 20: 0.5},
    16: {11: (0.25), 12: (0.25), 17: (0.25), 21: 0.25},
    17: {12: (0.25), 16: (0.25), 18: (0.25), 22: 0.25},
    18: {12: 0.2, 13: 0.2, 17: 0.2, 19: 0.2, 23: 0.2},
    19: {14: (1 / 3), 18: (1 / 3), 24: (1 / 3)},
    20: {15: 0.5, 21: 0.5},
    21: {16: (1 / 3), 20: (1 / 3), 22: (1 / 3)},
    22: {17: (1 / 3), 21: (1 / 3), 23: (1 / 3)},
    23: {18: (1 / 3), 22: (1 / 3), 24: (1 / 3)},
    24: {19: 0.5, 23: 0.5}
}

emission = {}
for i in range(25):
    emission[i] = {0: (1 / 11), 1: (1 / 11), 2: (1 / 11), 3: (1 / 11), 4: (1 / 11), 5: (1 / 11), 6: (1 / 11), 7: (1 / 11),
        8: (1 / 11), 9: (1 / 11), 10: (1 / 11)}
    
initial = {0: 0.04, 1: 0.04, 2: 0.04, 3: 0.04, 4: 0.04, 5: 0.04, 6: 0.04, 7: 0.04, 8: 0.04, 9: 0.04, 10: 0.04, 11: 0.04,
           12: 0.04, 13: 0.04, 14: 0.04, 15: 0.04, 16: 0.04, 17: 0.04, 18: 0.04, 19: 0.04, 20: 0.04, 21: 0.04, 22: 0.04,
           23: 0.04, 24: 0.04}



In [ ]:

model = HiddenMarkovModel(initial, transition, emission)
model.baum_welch(train_data)

output = json.dumps(model.transition_probs)
f = open("transition_probs.json", "w")
f.write(output)
f.close()

output = json.dumps(model.emission_probs)
f = open("emission_probs.json", "w")
f.write(output)
f.close()

output = json.dumps(model.initial_probabilities)
f = open("initial_probs.json", "w")
f.write(output)
f.close()

In [278]:
from pathlib import Path
initial_converged = json.loads(Path('initial_probs.json').read_text())
transition_converged = json.loads(Path('transition_probs.json').read_text())
emission_converged = json.loads(Path('emission_probs.json').read_text())

initial_converged = {int(k): v for k, v in initial_converged.items()}
transition_converged = {int(k): {int(x): z for x, z in v.items()} for k, v in transition_converged.items()}
emission_converged = {int(k): {int(x): z for x, z in v.items()} for k, v in emission_converged.items()}

model = HiddenMarkovModel(initial_converged,transition_converged,emission_converged)

# Q2

In [275]:
test_data= np.load('test_data.npy')

with open('forward_backward_output.txt', 'w') as f:
    for test in tqdm(test_data):
        f_prob = model.likelihood(test, log=True)
        b_prob = model.backward_likelihood(test, log=True)
        print(f_prob, "\t", np.exp(f_prob))
        print(b_prob, "\t", np.exp(b_prob))
        print("--------------------")
        f.write(str(np.exp(f_prob))+"\t"+str(np.exp(b_prob))+"\n")

  2%|▏         | 2/100 [00:00<00:13,  7.21it/s]

-87.40822221594169 	 1.0941866674560593e-38
-87.40822221594168 	 1.0941866674560748e-38
--------------------
-96.61173593224723 	 1.1016818575067096e-42
-96.61173593224727 	 1.1016818575066626e-42
--------------------


  4%|▍         | 4/100 [00:00<00:12,  7.44it/s]

-92.86014462227607 	 4.691930838968532e-41
-92.8601446222761 	 4.691930838968398e-41
--------------------
-94.30465596383002 	 1.1066449260434495e-41
-94.30465596383004 	 1.106644926043418e-41
--------------------


  6%|▌         | 6/100 [00:00<00:12,  7.41it/s]

...

-91.24412018883183 	 2.3614681685546735e-40
-91.24412018883181 	 2.361468168554707e-40
--------------------
-90.69811940737132 	 4.076685628753131e-40
-90.69811940737128 	 4.076685628753305e-40
--------------------
-96.87895765951693 	 8.433411191589496e-43
-96.87895765951696 	 8.433411191589257e-43
--------------------


 99%|█████████▉| 99/100 [00:09<00:00, 12.81it/s]

-95.96284038104089 	 2.10798712865435e-42
-95.96284038104089 	 2.10798712865435e-42
--------------------
-90.09279498686587 	 7.467861897169648e-40
-90.0927949868658 	 7.46786189717018e-40
--------------------
-100.38579699711615 	 2.529311407187092e-44
-100.3857969971162 	 2.529311407186948e-44
--------------------


100%|██████████| 100/100 [00:10<00:00,  9.93it/s]

-89.59896282555579 	 1.223671543435187e-39
-89.59896282555573 	 1.2236715434352567e-39
--------------------


In [277]:
test_data = np.load('test_data.npy')

with open('viterbi_output.txt', 'w') as f:
    for test in tqdm(test_data):
        s = model.viterbi(test)
        print(s)
        print("======================")
        f.write(str(s)+"\n")



  8%|▊         | 8/100 [00:00<00:02, 37.02it/s]

[16, 11, 12, 16, 12, 18, 17, 18, 17, 22, 23, 22, 23, 22, 21, 16, 21, 20, 21, 22, 23, 22, 21, 22, 23, 22, 17, 12, 8, 9, 4, 3, 2, 3, 4, 3, 4, 3, 8, 7, 8, 13, 12, 11, 12, 8, 7, 8, 3, 2]
[16, 21, 16, 12, 17, 12, 11, 16, 21, 22, 17, 12, 16, 11, 12, 6, 11, 16, 11, 12, 11, 6, 5, 10, 5, 6, 11, 12, 16, 21, 20, 15, 20, 21, 16, 21, 20, 15, 20, 21, 16, 21, 16, 12, 6, 5, 6, 5, 6, 12]
[21, 16, 12, 11, 6, 5, 6, 11, 6, 11, 6, 7, 8, 7, 8, 12, 17, 12, 11, 12, 7, 8, 9, 4, 3, 4, 9, 8, 12, 16, 21, 20, 21, 20, 15, 20, 21, 16, 12, 17, 12, 18, 17, 22, 17, 12, 13, 18, 17, 18]
[17, 12, 11, 12, 11, 12, 16, 21, 16, 21, 16, 11, 12, 17, 22, 23, 22, 17, 12, 11, 12, 6, 7, 8, 13, 14, 19, 14, 19, 14, 13, 12, 16, 21, 16, 11, 12, 6, 11, 12, 8, 7, 8, 13, 12, 18, 19, 18, 19, 14]
[21, 16, 11, 6, 7, 6, 12, 18, 17, 18, 17, 12, 18, 12, 6, 11, 16, 21, 16, 12, 11, 6, 5, 6, 5, 10, 5, 0, 5, 10, 5, 10, 5, 10, 5, 0, 1, 2, 1, 0, 1, 2, 1, 0, 1, 0, 1, 0, 1, 0]
[20, 21, 16, 21, 22, 23, 22, 23, 22, 17, 18, 19, 18, 19, 18, 17, 18, 12, 18,

 16%|█▌        | 16/100 [00:00<00:02, 37.65it/s]

[19, 14, 13, 8, 3, 2, 1, 0, 1, 0, 1, 2, 1, 0, 1, 0, 5, 0, 1, 0, 1, 0, 1, 2, 3, 2, 1, 2, 3, 8, 9, 4, 9, 8, 12, 13, 8, 9, 4, 9, 14, 13, 14, 13, 14, 19, 24, 23, 22, 23]
[5, 0, 1, 2, 3, 4, 9, 8, 12, 17, 18, 19, 18, 17, 22, 23, 22, 23, 22, 23, 22, 23, 22, 17, 22, 17, 22, 23, 22, 23, 22, 23, 22, 17, 12, 6, 5, 6, 7, 8, 7, 6, 12, 8, 7, 6, 7, 6, 11, 12]
[15, 20, 15, 20, 15, 20, 15, 20, 21, 22, 23, 22, 23, 22, 17, 18, 17, 22, 17, 22, 23, 22, 17, 18, 12, 16, 12, 13, 18, 19, 14, 19, 18, 12, 17, 22, 23, 24, 19, 24, 23, 22, 17, 18, 17, 12, 16, 12, 16, 21]
[4, 3, 2, 1, 0, 1, 2, 1, 0, 1, 0, 1, 2, 1, 2, 1, 2, 3, 4, 3, 4, 3, 8, 13, 12, 18, 17, 22, 17, 22, 23, 22, 23, 22, 23, 22, 23, 22, 23, 22, 17, 18, 12, 16, 11, 6, 5, 10, 5, 6]
[15, 20, 15, 20, 21, 20, 21, 20, 15, 20, 21, 16, 11, 6, 5, 10, 5, 6, 11, 12, 11, 16, 21, 20, 21, 16, 11, 12, 18, 12, 11, 16, 11, 12, 18, 17, 22, 23, 22, 21, 20, 15, 20, 15, 20, 15, 20, 15, 20, 15]
[6, 11, 12, 18, 17, 18, 19, 14, 19, 24, 23, 22, 23, 22, 17, 18, 19, 24, 23, 22, 2

 24%|██▍       | 24/100 [00:00<00:02, 37.39it/s]

...

[13, 14, 19, 18, 17, 18, 19, 14, 19, 18, 12, 18, 17, 22, 21, 22, 23, 22, 17, 18, 17, 18, 17, 22, 21, 16, 12, 11, 6, 5, 10, 5, 10, 5, 10, 5, 10, 5, 6, 12, 11, 16, 21, 20, 15, 20, 15, 20, 15, 20]
[16, 11, 12, 18, 19, 14, 13, 12, 7, 8, 3, 2, 3, 2, 1, 2, 3, 4, 3, 2, 3, 8, 12, 7, 8, 12, 13, 18, 17, 22, 17, 16, 12, 18, 19, 18, 17, 22, 17, 16, 12, 11, 16, 21, 16, 12, 13, 8, 7, 8]
[13, 14, 19, 24, 23, 24, 19, 18, 17, 12, 11, 12, 11, 6, 7, 8, 7, 6, 12, 11, 12, 16, 21, 16, 21, 16, 11, 12, 11, 16, 12, 13, 8, 7, 8, 3, 4, 9, 14, 19, 24, 23, 22, 23, 22, 17, 22, 23, 24, 19]
[2, 3, 8, 9, 8, 3, 4, 3, 2, 1, 0, 1, 0, 1, 2, 1, 0, 1, 0, 1, 2, 3, 2, 3, 8, 12, 17, 12, 11, 16, 17, 18, 12, 11, 12, 18, 19, 18, 12, 17, 22, 17, 12, 18, 17, 22, 23, 22, 17, 12]
[19, 24, 23, 22, 23, 22, 17, 18, 17, 18, 12, 8, 7, 6, 7, 6, 7, 6, 11, 12, 18, 12, 17, 22, 17, 12, 18, 17, 18, 19, 14, 19, 14, 19, 24, 23, 22, 23, 22, 17, 18, 19, 18, 17, 12, 7, 8, 9, 8, 3]
[4, 9, 8, 3, 4, 3, 4, 3, 4, 9, 8, 9, 4, 3, 2, 1, 2, 1, 0, 1, 2, 1, 2,

 89%|████████▉ | 89/100 [00:02<00:00, 39.21it/s]

[22, 17, 18, 19, 18, 19, 18, 17, 18, 19, 18, 17, 18, 19, 24, 23, 22, 17, 18, 12, 16, 11, 12, 11, 6, 5, 6, 5, 10, 5, 10, 5, 6, 11, 12, 18, 17, 22, 23, 22, 23, 22, 23, 22, 21, 16, 12, 11, 12, 16]
[2, 1, 0, 1, 2, 1, 0, 5, 6, 5, 10, 5, 10, 5, 6, 5, 6, 5, 10, 5, 0, 5, 0, 1, 0, 1, 2, 1, 0, 5, 6, 5, 0, 1, 2, 1, 0, 5, 6, 12, 8, 9, 8, 12, 17, 18, 17, 12, 16, 21]
[5, 10, 5, 10, 5, 0, 1, 2, 3, 2, 3, 2, 1, 2, 3, 4, 3, 8, 3, 4, 3, 4, 9, 8, 12, 16, 11, 12, 18, 12, 16, 11, 12, 13, 8, 7, 6, 12, 11, 16, 21, 20, 21, 16, 12, 7, 8, 7, 6, 5]
[15, 20, 15, 20, 15, 20, 15, 20, 15, 20, 15, 20, 15, 20, 15, 20, 15, 20, 15, 20, 15, 20, 15, 20, 15, 20, 15, 20, 21, 22, 23, 22, 23, 22, 23, 22, 23, 22, 21, 20, 21, 16, 21, 20, 15, 20, 15, 20, 15, 20]
[10, 5, 6, 5, 6, 5, 6, 5, 6, 11, 16, 21, 16, 12, 16, 21, 16, 12, 6, 11, 6, 5, 6, 12, 13, 8, 9, 4, 3, 2, 3, 8, 7, 8, 7, 8, 7, 12, 16, 21, 16, 21, 20, 21, 22, 23, 22, 17, 12, 16]
[5, 10, 5, 6, 12, 18, 12, 18, 12, 7, 8, 12, 13, 14, 13, 12, 16, 21, 20, 21, 20, 21, 16, 12, 13,

100%|██████████| 100/100 [00:02<00:00, 39.40it/s]

[14, 19, 24, 23, 22, 23, 24, 19, 18, 17, 18, 17, 22, 17, 12, 16, 12, 18, 17, 22, 21, 16, 12, 11, 16, 12, 7, 8, 12, 6, 11, 12, 8, 13, 14, 19, 24, 23, 24, 19, 18, 17, 12, 11, 6, 7, 8, 7, 6, 11]
[7, 6, 7, 8, 13, 12, 6, 7, 8, 12, 17, 18, 12, 17, 22, 23, 22, 17, 22, 23, 22, 17, 12, 13, 14, 13, 8, 12, 18, 17, 18, 12, 11, 12, 17, 18, 17, 18, 17, 18, 12, 18, 12, 11, 12, 11, 12, 7, 8, 12]
[12, 18, 17, 18, 19, 24, 23, 22, 23, 24, 19, 14, 9, 4, 9, 8, 13, 18, 19, 24, 23, 18, 17, 22, 23, 22, 17, 18, 17, 12, 16, 21, 20, 15, 20, 15, 20, 21, 20, 21, 20, 15, 20, 15, 20, 15, 20, 15, 20, 15]
[23, 22, 17, 22, 17, 12, 16, 21, 16, 21, 16, 12, 16, 21, 16, 21, 16, 12, 17, 22, 21, 20, 21, 16, 21, 16, 11, 16, 12, 11, 6, 5, 6, 7, 8, 13, 12, 18, 19, 24, 23, 22, 17, 22, 23, 24, 19, 18, 12, 6]
[5, 10, 5, 0, 5, 10, 5, 6, 7, 8, 7, 8, 3, 2, 1, 0, 1, 0, 1, 2, 1, 2, 1, 2, 3, 2, 1, 0, 1, 0, 5, 0, 1, 2, 3, 4, 9, 8, 7, 8, 13, 14, 19, 24, 23, 22, 17, 12, 16, 21]
[19, 14, 19, 14, 19, 14, 13, 8, 12, 17, 22, 23, 22, 23, 22, 21

# Question 4

In [168]:
def convert_dictionary_to_matrix(dictionary):
    rows=len(dictionary)
    matrix=np.zeros((rows,rows),dtype=np.float64)
    for i in range(rows):
        for j in range(rows):
            value = dictionary[i].get(j)
            matrix[i,j] = value if value is not None else 0
    return matrix

In [169]:
def get_stationary_distribution(Q):
    evals, evecs = np.linalg.eig(Q.T)
    evec1 = evecs[:,np.isclose(evals, 1)]
    evec1 = evec1[:,0]
    stationary = evec1 / evec1.sum()
    stationary = stationary.real
    stationary = stationary / stationary.sum()
    return stationary

In [170]:
def remove_edge(dictionary, left, right):
    dictionary[left].pop(right)
    dictionary[right].pop(left)

In [171]:
def normalize_matrix(matrix):
    """
    input: two dimensional matrix
    returns a matrix which its rows sum to 1
    """
    return matrix / np.sum(matrix,1)[:,None]

In [216]:
import copy
transition_matrix = convert_dictionary_to_matrix(model.transition_probs)
transition2 =copy.deepcopy(model.transition_probs)

remove_edge(transition2,8,12)
remove_edge(transition2,12,13)
remove_edge(transition2,18,19)
remove_edge(transition2,16,17)
remove_edge(transition2,6,12)

transition_matrix2 = convert_dictionary_to_matrix(transition2)
transition_matrix2 = normalize_matrix(transition_matrix2)

stationary_distribution=get_stationary_distribution(transition_matrix)
stationary_distribution

array([0.02112507, 0.0285067 , 0.02265319, 0.01969918, 0.01347683,
       0.03876496, 0.04224324, 0.02300952, 0.03733263, 0.01653681,
       0.01811561, 0.04662704, 0.11368048, 0.02230126, 0.01947599,
       0.02869246, 0.07049459, 0.07837297, 0.05633585, 0.03169953,
       0.0491273 , 0.05711506, 0.08066256, 0.05072891, 0.01322225])

In [180]:
from random import uniform
import random
import pandas as pd
def w_choice(seq):
    """
    choose an item from list of items whose probability is given
    example: input [0.3,0.4,0.3] 
    output: it may choose from [0,1,2] with given probabilty
    """
    total_prob = sum([item for item in seq])
    chosen = random.uniform(0,total_prob)
    cumulative = 0 
    for i in range(len(seq)):
        cumulative += seq[i]
        if cumulative > chosen:
            return i

In [181]:
state_count = len(transition_matrix)
a_array = np.zeros((state_count,state_count))
for i in range(state_count):
    for j in range(state_count):
        r_nominator = stationary_distribution[j]*transition_matrix2[j,i] 
        r_denominator = stationary_distribution[i]*transition_matrix2[i,j]

        a = 0
        if r_denominator == 0:
            a = 1
        else:
            a = min(r_nominator / r_denominator,1)
            
        a_array[i,j] = a

In [182]:
new_transition_matrix=a_array*transition_matrix2
stationary2= get_stationary_distribution(normalize_matrix(new_transition_matrix))
stationary2

array([0.02180875, 0.02994216, 0.0234925 , 0.02023764, 0.01302789,
       0.04241142, 0.03536303, 0.02535294, 0.03216247, 0.01639951,
       0.01965901, 0.0513758 , 0.10217224, 0.02017778, 0.02124298,
       0.03131307, 0.07050133, 0.08362803, 0.05162414, 0.01804102,
       0.05382912, 0.06143372, 0.08667532, 0.05355925, 0.01456888])

In [241]:
### check if they are close
(stationary2-stationary_distribution) < 0.01

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [100]:
### sampling
iterations = 1000
sample_items = 100
x_array = np.zeros((iterations,sample_items))
for e in range(iterations):
    x_array[e,0] = int(random.uniform(0,state_count))
    for i in range(sample_items-1):
        x = int(x_array[e,i])
        y = w_choice(transition_matrix2[x])
        
        if random.uniform(0,1) <= a_array[x,y]:
            x_array[e,i+1] = y
        else:
            x_array[e,i+1] = x

In [113]:
x_array=x_array.reshape(-1)
imperical_stationary=pd.Series(x_array).value_counts().sort_index()/len(x_array)
np.array(imperical_stationary)-stationary_distribution < 0.01

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [114]:
stationary2= get_stationary_distribution(normalize_matrix(new_transition_matrix))
stationary2

array([0.02180875, 0.02994216, 0.0234925 , 0.02023764, 0.01302789,
       0.04241142, 0.03536303, 0.02535294, 0.03216247, 0.01639951,
       0.01965901, 0.0513758 , 0.10217224, 0.02017778, 0.02124298,
       0.03131307, 0.07050133, 0.08362803, 0.05162414, 0.01804102,
       0.05382912, 0.06143372, 0.08667532, 0.05355925, 0.01456888])

In [115]:
(stationary2-stationary_distribution) < 0.01

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

### Question 5

In [157]:
min_matrix=np.where(transition_matrix<transition_matrix.T,transition_matrix,transition_matrix.T)
proposal_stationary=get_stationary_distribution(normalize_matrix(min_matrix))
proposal_stationary

array([0.03623193, 0.05314085, 0.04338589, 0.039074  , 0.04035088,
       0.05016627, 0.0429694 , 0.02380878, 0.04097378, 0.04377247,
       0.0249732 , 0.03266443, 0.05314085, 0.02670919, 0.03966445,
       0.03096373, 0.04686786, 0.04565732, 0.03981049, 0.04184281,
       0.04970957, 0.04558968, 0.05314085, 0.03842504, 0.01696628])

> Yes, It is logical, 

# Extra Section for testing purposes

In [191]:
import numpy as np
transition = np.array([[0.1,0.2,0.7],
					[0.3,0.6,0.1],
					[0.3,0.4,0.3] ])

In [192]:
np.linalg.matrix_power(transition,10000)

array([[0.25  , 0.4375, 0.3125],
       [0.25  , 0.4375, 0.3125],
       [0.25  , 0.4375, 0.3125]])

In [193]:
stationary_dist=get_stationary_distribution(transition)
stationary_dist

array([0.25  , 0.4375, 0.3125])

In [205]:
def metropolis_hastings(sample_items, transition, stationary_dist):
    sample = np.zeros(sample_items,dtype=np.int32)
    for i in range(sample_items-1):
            x = sample[i]
            y = w_choice(transition[x])

            r_nominator = stationary_dist[y]*transition[y,x] 
            r_denominator = stationary_dist[x]*transition[x,y]

            a = 0
            if r_denominator == 0:
                a = 1
            else:
                a = min(r_nominator / r_denominator,1)

            if random.uniform(0,1) <= a:
                sample[i+1] = y
            else:
                sample[i+1] = x
    return sample
print(metropolis_hastings(10,transition,stationary_dist))

[0 0 0 1 1 2 0 0 2 0]


In [213]:
import pandas as pd
number_of_iterations = 100000
samples_per_iteration = 100
samples=np.zeros((number_of_iterations,samples_per_iteration))
for i in range(number_of_iterations):
    samples[i]= metropolis_hastings(samples_per_iteration,transition,stationary_dist)
    
samples=samples.reshape(-1)
imperical_stationary=pd.Series(samples).value_counts().sort_index()/len(samples)
imperical_stationary

0.0    0.260744
1.0    0.427493
2.0    0.311763
dtype: float64